In [1]:
import mariadb
import sys
import datetime

#Encryption Stuff
import base64
from Crypto.Cipher import AES
import http.client
from Crypto import Random

#Other
import json
import time
from datetime import timedelta

In [2]:
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="airq",
        password="airq",
        host="localhost",
        port=3306,
        database="airq_data"
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [3]:
# Verbindungsinformationen für den Air-q Sensor
airqIP = '192.168.4.1'
airqpass = 'airqsetup'


def unpad(data):
    return data[:-ord(data[-1])]


def decodeMessage(msgb64):
    # Erster Schritt: base64 dekodieren
    msg = base64.b64decode(msgb64)

    # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
    key = airqpass.encode('utf-8')
    if len(key) < 32:
        for i in range(32 - len(key)):
            key += b'0'
    elif len(key) > 32:
        key = key[:32]

    # Zweiter Schritt: AES256 dekodieren
    cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=msg[:16])
    return unpad(cipher.decrypt(msg[16:]).decode('utf-8'))


def pad(data):
    length = 16 - (len(data) % 16)
    return data + chr(length).encode('utf-8') * length


def encodeMessage(msg):
    # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
    key = airqpass.encode('utf-8')
    if len(key) < 32:
        for i in range(32 - len(key)):
            key += b'0'
    elif len(key) > 32:
        key = key[:32]

    # Erster Schritt: AES256 verschlüsseln
    iv = Random.new().read(AES.block_size)
    cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=iv)
    msg = msg.encode('utf-8')
    crypt = iv + cipher.encrypt(pad(msg))

    # Zweiter Schritt: base64 enkodieren
    msgb64 = base64.b64encode(crypt).decode('utf-8')
    return msgb64

In [4]:
#Funktion zur Erstellung des SQL-Statements und Speicherung in der Datenbank
def sql_data(contents, cur, table_name):
    # content wird Zeile für Zeile verarbeitet
    for line in contents.read().split(b'\n'):
        if line != b'':
            # Message wird dekodiert
            line = decodeMessage(line)
            # print(line)

            # Zeile wird in ein dict konvertiert
            line = json.loads(line)

            columns = ""
            values = ""

            if line["Status"] != "OK":
                print(line["Status"])
                print("Skipped invalid measurements due to warm-up of the Sensor")
                continue

            for type in line:
                # Diese Daten werden herausgefiltert und nicht mit in die Datenbank mit aufgenommen
                if type == "bat" or type == "DeviceID" or type == "uptime" or type == "window_event" or type == "door_event" or type == "person" or type == "window_open" or type == "Status":
                    continue

                # Erstellt den String für die Columns
                columns += type

                # Erstellt den String für die Values
                # Edge-case: wenn value eine Liste ist
                if isinstance(line[type], list):
                    values += str(line[type][0])
                elif type == "timestamp":
                    values += "FROM_UNIXTIME(%s)" % (int(line[type] / 1000))
                else:
                    values += str(line[type])

                # Edge-case: Am Ende kein Komma
                if type != "cnt0_3":
                    columns += ", "
                    values += ", "

            sql = f"INSERT IGNORE INTO {table_name} ( {columns} ) VALUES ( {values} );"
            print(sql)
            cur.execute(sql)
            conn.commit()

In [5]:
def file_to_db(start_date, end_date, table_name):
    start_datetime = datetime.datetime.strptime(start_date, "%Y/%m/%d")
    end_datetime = datetime.datetime.strptime(end_date, "%Y/%m/%d")
    current_datetime = start_datetime

    while current_datetime <= end_datetime:
        current_date = current_datetime.strftime("%Y/%m/%d")
        connection = http.client.HTTPConnection(airqIP)
        connection.request("GET", "/file?request=" + encodeMessage(current_date))
        contents = connection.getresponse()

        if contents.status == 200:
            print("Status: " + str(contents.status) + "/OK")
            sql_data(contents, cur, table_name)
        else:
            print("Status: " + str(contents.status) + "/Canceled!")

        connection.close()
        current_datetime += datetime.timedelta(days=1)


In [6]:
# Beispielaufruf der Funktion
start_date = "2023/05/23"
end_date = "2023/06/13"
table_name = "measurements_1"

file_to_db(start_date, end_date, table_name)

# Close the database connection
conn.close()

Status: 200/OK


JSONDecodeError: Expecting value: line 1 column 1 (char 0)